In [2]:
# This notebook is to build the code to scrape the Toronto postal service Wikipedia page

# download and importing libraries
!pip install BeautifulSoup4
!pip install requests
!pip install lxml

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests


In [ ]:
#### Now import the dataset from the wikipedia page

In [3]:
toronto_wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(toronto_wiki).text

soup = BeautifulSoup(source, 'lxml')

In [4]:
#get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
toronto_df = pd.DataFrame(data = data,columns = columns)
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
#Remove Boroughs that are 'Not assigned'

toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.head()


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [17]:
#print the shape and size

print("Shape: ", toronto_df.shape)

Shape:  (103, 3)
